# Landsat NDVI Processing with Google Earth Engine

This notebook processes Landsat 8 imagery to calculate median NDVI values for specific years and exports the results to Earth Engine assets.

In [ ]:
import ee

# Initialize Google Earth Engine
ee.Initialize()

In [ ]:
# Define the parameters
years = [2013, 2015, 2017, 2019, 2021, 2023]

# Define the area of interest (feature collection)
aoi = ee.FeatureCollection('projects/ee-komba/assets/bbox_wirong')

# Define the time range (April-June)
start_date = '04-01'
end_date = '06-30'

In [ ]:
def calculate_and_export_median_ndvi(year):
    # Filter the image collection by date and area
    filtered_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(ee.Date.fromYMD(year, 4, 1), ee.Date.fromYMD(year, 6, 30)) \
        .filterBounds(aoi)

    def mask_clouds_and_calculate_ndvi(image):
        # Apply cloud mask
        cloud_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)  
        masked_image = image.updateMask(cloud_mask)
        # Calculate NDVI
        ndvi = masked_image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')  
        return masked_image.addBands(ndvi)

    # Apply cloud masking and NDVI calculation to each image in the collection
    processed_collection = filtered_collection.map(mask_clouds_and_calculate_ndvi)

    # Calculate the median NDVI
    median_ndvi = processed_collection.select('NDVI').median()

    # Define the export path
    export_path = f'projects/ee-komba/assets/kaya/median_ndvi_{year}'

    # Export the median NDVI as an asset
    task = ee.batch.Export.image.toAsset(
        image=median_ndvi,
        description=f'median_ndvi_{year}',
        assetId=export_path,
        region=aoi.geometry().bounds(),
        scale=30,  # Adjust the scale as needed
        maxPixels=1e13  # Adjust maxPixels as needed
    )
    task.start()
    print(f'Exporting median NDVI for {year} to {export_path}')

In [ ]:
# Loop through the years and calculate/export median NDVI
for year in years:
    calculate_and_export_median_ndvi(year)

print('All tasks started.')